# Sieci splotowe

In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## Wprowadzenie
### Przygotowanie danych
Ponownie wykorzystamy w zadaniu zbiór MNIST. Zaczynamy od pobrania dnaych.

In [ ]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')
print(mnist.data.shape)

Każda obraz przedstawiony jest jako wektor 784 liczb, odpowiadających kolejnym pikselom. Żeby wykorzystać splotowe sieci neuronowe musimy przejść do reprezentacji trójwymiarowej: szerokosc $\times$ wysokość $\times$ kanały. Przekształcamy zatem macierz w czterowymiarowy tensor tak, żeby każdy obraz był trójwymiarowym tensorem $28\times 28\times 1$. Przy okazji dokonamy normalizacji tak, żeby wszystkie wartości były z przedziału 0-1.

In [ ]:
X = mnist.data.reshape((-1,28,28,1))
X = X / np.max(X)

Dzielimy dane na zbiór uczący, walidujący i testowy.

In [ ]:
n, p = mnist.data.shape
k = 10 # liczba klas
n_train = int(.7*n)
n_validation = int(.1*n)
indices = np.random.permutation(n)
train_indices = indices[:n_train]
validation_indices = indices[n_train:n_train+n_validation]
test_indices = indices[n_train+n_validation:]
X_train, y_train = X[train_indices,:,:], mnist.target[train_indices]
X_validation, y_validation = X[validation_indices,:,:], mnist.target[validation_indices]
X_test, y_test = X[test_indices,:], mnist.target[test_indices]

### Budowa sieci neuronowej

Rozpoczynamy od przygotowania placeholderów na dane

In [ ]:
tf.reset_default_graph()
X_pl = tf.placeholder(dtype=tf.float32, shape=(None, 28, 28, 1))
y_pl = tf.placeholder(dtype=tf.int64, shape=(None,))

Pierwsza warstwa naszej sieci będzie składała się z 6 filtrów rozmiaru $3\times 3\times 1$, na których uruchomiona zostanie funkcja aktywacji ELU. Zmienna `filters` będzie zawierała wagi tych filtrów. Wykorzystamy krok 1 we wszystkich wymiarach (parametr `stride`) i uzupełnianie `VALID`, tzn. przekształcone obrazy będą mniejsze o 2 piksele z każdej strony.

In [ ]:
filters = tf.get_variable(name='layer1_filters', shape=(3,3,1,6))
layer1_intermediate = tf.nn.conv2d(X_pl, filters, strides=[1,1,1,1], padding='VALID')
layer1 = tf.nn.elu(layer1_intermediate)

Wyświetlmy rozmiar i sprawdźmy czy wszystko się zgadza

In [ ]:
print(layer1.shape)

Nastepnie wykonamy pooling i zmniejszymy rozmiar do $12\times 12\times 6$

In [ ]:
layer2_intermediate = tf.nn.avg_pool(layer1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
layer2 = tf.nn.elu(layer2_intermediate)

In [ ]:
print(layer2.shape)

Utworzymy warstwę wyjściową: 10 neuronów, każdy z $12\cdot 12\cdot 6=864$ wejściami

In [ ]:
layer3_input = tf.reshape(layer2, shape=(-1, 12*12*6))
logits = tf.layers.dense(layer3_input, 10, activation=None)

Przygotowujemy kolejno: 

* optymalizator do uczenia
* miarę jakości
* inicjalizator zmiennych i obiekt sesji

In [ ]:
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y_pl))
minimizer = tf.train.AdamOptimizer().minimize(loss)

In [ ]:
y_pred = tf.argmax(logits, axis=1)
correct = tf.equal(y_pred, y_pl)
correct = tf.cast(correct, dtype=tf.float32)
acc = tf.reduce_mean(correct)

In [ ]:
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()

Inicjalizujemy zmienne, uczymy przez 1000 epok używając zbiorów po 100 przykładów, rysujemy wykres straty w czasie, obliczamy accuracy na zbiorze walidującym

In [ ]:
sess.run(init)
loss_values = []

In [ ]:
for i in range(1000):
    indices = np.random.choice(n_train, size=100) 
    feed_dict = {X_pl: X_train[indices,...], y_pl: y_train[indices]}
    _, loss_val = sess.run([minimizer, loss], feed_dict)
    loss_values.append(loss_val)

In [ ]:
plt.plot(loss_values)
plt.show()

In [ ]:
feed_dict = {X_pl: X_validation, y_pl: y_validation}
print(sess.run(acc, feed_dict))

# Zadanie do samodzielnego wykonania

Zaimplementuj sieć o architekturze zbliżonej do LeNet-5, zgodnie z poniższym opisem:

Nr warstwy|Typ|Rozmiar wyjścia|Liczba filtrów|Rozmiar jądra (`ksize`)|Krok (`stride`)|Funkcja aktywacji|
--- | --- | ---
0|wejście|$32\times 32$|obrazki trzeba najpierw uzupełnić na krawędziach zerami
1|splotowa|$28\times 28$|6|$5\times 5$|1|tanh
2|avgerage pooling|$14\times 14$|6|$2\times 2$|2|tanh
3|splotowa|$10\times 10$|16|$5\times 5$|1|tanh
4|average pooling|$5\times 5$|16|$2\times 2$|2|tanh
5|splotowa|$1\times 1$|120|$5\times 5$|1|tanh
6|pełna|84||||tanh
7|pełna|10||||brak

## Zadania szczegółowe:
1. Uzupełnij obrazki zerami
2. Podziel dane na trzy podzbiory
3. Zbuduj sieć
4. Ucz sieć wykorzystując early stopping
5. Oceń jakość sieci na zbiorze testowym
6. Wyświetl 10 źle zaklasyfikowanych przypadków ze zbioru testowego